In [ ]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-2.0-flash-001")

# Structured output

Let's define the data structure that describes a plan to solve a complex task:

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate


class Step(BaseModel):
    """A step that is a part of the plan to solve the task."""
    step: str = Field(description="Description of the step")


class Plan(BaseModel):
    """A plan to solve the task."""
    steps: list[Step]


prompt = PromptTemplate.from_template(
    "Prepare a step-by-step plan to solve the given task.\n"
    "TASK:\n{task}\n"
)

We can explore that the model has succcessfully generated a complex _Pydantic_ structure, and all we needed to do was using a `with_structured_output` method:

In [ ]:
result = (prompt | llm.with_structured_output(Plan)).invoke("How to write a bestseller on Amazon about generative AI?")
assert isinstance(result, Plan)
print(f"Amount of steps: {len(result.steps)}")
for step in result.steps:
  print(step.step)
  break

Amount of steps: 5
**1. Idea Generation and Validation:**
*   **Brainstorm Unique Angles:**  Don't just rehash existing content. Find a fresh perspective on generative AI. Consider focusing on specific applications (e.g., AI in art, business, healthcare), ethical implications, future trends, or personal stories of transformation through AI.
*   **Keyword Research:** Use tools like Ahrefs, SEMrush, or even Amazon's search bar to identify relevant keywords and search volume. This will help you understand market demand and optimize your book for discoverability.
*   **Competitive Analysis:** Study existing bestsellers on Amazon about AI. What are their strengths and weaknesses? What gaps can you fill? How can you differentiate your book?
*   **Target Audience Definition:** Clearly define your ideal reader. What are their interests, needs, and pain points?  Knowing your audience will guide your writing style and content.


We can also use a `json_mode` and pass a custom schema to an LLM. 

In [ ]:
plan_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
          "properties": {
              "step": {"type": "STRING"},
          },
      },
}

query = "How to write a bestseller on Amazon about generative AI?"
result = (prompt | llm.with_structured_output(schema=plan_schema, method="json_mode")).invoke(query)

In [ ]:
assert(isinstance(result, list))
print(f"Amount of steps: {len(result)}")
print(result[0])

Amount of steps: 13
[{'step': 'Market Research: Analyze the existing books on generative AI on Amazon. Identify gaps, underserved niches, and potential angles.  Analyze best-selling books in similar categories (e.g., tech, AI, business) to understand successful marketing strategies and reader preferences.'}]


As an alternative, we can use custom arguments supported by the LLM provider. Please, note that these options are vendor-specific, and you need to check the corresponding documentatino for supported extra arguments and formats:

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

llm_json = ChatVertexAI(model_name="gemini-1.5-pro-002", response_mime_type="application/json", response_schema=plan_schema)
result = (prompt | llm_json | JsonOutputParser()).invoke(query)
assert(isinstance(result, list))
print(f"Amount of steps: {len(result)}")
print(result[0])

Amount of steps: 9
{'step': '1. Understand the Generative AI Landscape: Research the current state of generative AI, key players, emerging trends, and potential future developments. Identify the gaps in existing literature and target a specific niche within the generative AI field (e.g., creative writing, image generation, code generation, etc.).'}


We can also generated a enum (again, it's a vendor-dependent feature):

In [ ]:
from langchain_core.output_parsers import StrOutputParser

response_schema = {"type": "STRING", "enum": ["positive", "negative", "neutral"]}

prompt = PromptTemplate.from_template(
    "Classify the tone of the following customer's review:"
    "\n{review}\n"
)

review = "I like this movie!"
llm_enum = ChatVertexAI(model_name="gemini-1.5-pro-002", response_mime_type="text/x.enum", response_schema=response_schema)
result = (prompt | llm_enum | StrOutputParser()).invoke(review)
print(result)

positive
